In [ ]:
#use an older version of scipy for sparse matrix processing
!pip install q scipy==1.1.0
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats


     |████████████████████████████████| 31.2MB 143kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,)), ])
trainset = datasets.MNIST('./',download = True, train = True, transform = transform)
valset = datasets.MNIST('./',download= True, train= False, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64,shuffle = True)
valloader = torch.utils.data.DataLoader(valset, batch_size = 64, shuffle = True)

dataiter = iter(trainloader)
images, labels = dataiter.next()
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)


#define the model 
inputsize = 784
hiddensizes = [192,192,192]
outputsize = 10

criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0],-1)


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [ ]:

#set random seed for reproducibility
torch.manual_seed(0)
net = nn.Sequential(nn.Linear(inputsize,hiddensizes[0]),nn.ReLU(),nn.Linear(hiddensizes[0],hiddensizes[1]),nn.ReLU(),
                    nn.Linear(hiddensizes[1],hiddensizes[2]),nn.ReLU(),nn.Linear(hiddensizes[2],outputsize),nn.LogSoftmax(dim=1))
net.apply(init_weights)

w_init= list(net.parameters())
torch.save(net.state_dict(),'weights_i.pth')

print(net)
logps = (net(images))
loss = criterion(logps,labels)


optimizer = optim.SGD(net.parameters(),lr = 0.003, momentum = 0.9 )
time0 = time()

Sequential(
  (0): Linear(in_features=784, out_features=192, bias=True)
  (1): ReLU()
  (2): Linear(in_features=192, out_features=192, bias=True)
  (3): ReLU()
  (4): Linear(in_features=192, out_features=192, bias=True)
  (5): ReLU()
  (6): Linear(in_features=192, out_features=10, bias=True)
  (7): LogSoftmax()
)


In [ ]:
epochs =10
for e in range(epochs):
    running_loss = 0
    for images,labels in trainloader:
        images = images.view(images.shape[0],-1)
        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
#     else:
    print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
#     print(running_loss)
            
    print("\n Training time in minutes = ", (time()-time0)/60)
    
    torch.save(net.state_dict(),'weights_f.pth')

Epoch 0 - Training loss: 0.4231571603947675

 Training time in minutes =  0.2004963755607605
Epoch 1 - Training loss: 0.19631705909117517

 Training time in minutes =  0.39354347785313926
Epoch 2 - Training loss: 0.14519017852390054

 Training time in minutes =  0.5865230758984884
Epoch 3 - Training loss: 0.11486083212822898

 Training time in minutes =  0.7793372313181559
Epoch 4 - Training loss: 0.09523867852866713

 Training time in minutes =  0.9751845955848694
Epoch 5 - Training loss: 0.08360487874498022

 Training time in minutes =  1.167382291952769
Epoch 6 - Training loss: 0.0708090368492493

 Training time in minutes =  1.3599907954533894
Epoch 7 - Training loss: 0.06276638066318672

 Training time in minutes =  1.554373606046041
Epoch 8 - Training loss: 0.054600059537083974

 Training time in minutes =  1.7471601247787476
Epoch 9 - Training loss: 0.04968463800135635

 Training time in minutes =  1.9390368660291035


In [ ]:
#Projecting weights on affine spaces defined by the set P: {AW = AW_final} 
#Projection is done using L2 minimization w= argmin ||w-w_initial|| in P

#vectorize W
wf =torch.load('weights_f.pth')
wf1 = np.asarray(wf['0.weight'].resize(1,hiddensizes[0]*784))
wf2 = np.asarray(wf['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wf3 = np.asarray(wf['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wf4 = np.asarray(wf['6.weight'].resize(1,hiddensizes[2]*10))
wf_vec = np.concatenate((wf1,wf2,wf3,wf4),axis =1)
wf_vec = wf_vec.T
dim_w = wf_vec.shape[0]


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [ ]:
#testing
correct_count, all_count = 0, 0
for images,labels in valloader:
    for i in range(len(labels)):
        img = images[i].view(1, 784)
        with torch.no_grad():
            logps = net(img)


        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.numpy()[i]
        if(true_label == pred_label):
              correct_count += 1
        all_count += 1
print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))
np.save('model_accuracy3.npy',(correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.9718


In [ ]:
dim_w

226176

262144

In [ ]:
#w_init ,A run  1
from scipy import sparse
n_runs =2
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,226176])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.104

 Dist from Wi = 0.030893499596637605 , Dist from Wf =7.962031145492706

 n_rows = 4 , Accuracy =0.1038

 Dist from Wi = 0.03375894784765674 , Dist from Wf =7.962019511628012

 n_rows = 8 , Accuracy =0.1038

 Dist from Wi = 0.04251510653143938 , Dist from Wf =7.961977570668239

 n_rows = 16 , Accuracy =0.1037

 Dist from Wi = 0.058751365438187333 , Dist from Wf =7.961874317469273

 n_rows = 32 , Accuracy =0.1038

 Dist from Wi = 0.07734016349876377 , Dist from Wf =7.9617154476358065

 n_rows = 64 , Accuracy =0.1033

 Dist from Wi = 0.13194841605770366 , Dist from Wf =7.960997675259366

 n_rows = 128 , Accuracy =0.1036

 Dist from Wi = 0.18341775402798377 , Dist from Wf =7.959978159357808

 n_rows = 256 , Accuracy =0.1051

 Dist from Wi = 0.2692252517416035 , Dist from Wf =7.957538069913441

 n_rows = 512 , Accuracy =0.1059

 Dist from Wi = 0.3809267796997006 , Dist from Wf =7.952973604819484

 n_rows = 1024 , Accuracy =0.1098

 D

In [ ]:
np.save("accuracy_A.npy", acc)
np.save("distance_i_A.npy", dist_i)
np.save("distance_f_A.npy", dist_f)

In [ ]:
#winit, AI run 1
I = sparse.identity(dim_w)
n_runs =2
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,226176])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs ) + I

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:] 
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.1037

 Dist from Wi = 0.01298093745528664 , Dist from Wf =7.962080498552496

 n_rows = 4 , Accuracy =0.1039

 Dist from Wi = 0.019348839186037477 , Dist from Wf =7.962067570211355

 n_rows = 8 , Accuracy =0.1036

 Dist from Wi = 0.0300872558696794 , Dist from Wf =7.962034232984789

 n_rows = 16 , Accuracy =0.104

 Dist from Wi = 0.05327114273346675 , Dist from Wf =7.961912870382221

 n_rows = 32 , Accuracy =0.1043

 Dist from Wi = 0.10437166589570834 , Dist from Wf =7.9614069689689115

 n_rows = 64 , Accuracy =0.104

 Dist from Wi = 0.1264250629475433 , Dist from Wf =7.961087304733998

 n_rows = 128 , Accuracy =0.1041

 Dist from Wi = 0.18503634898140484 , Dist from Wf =7.959940698255075

 n_rows = 256 , Accuracy =0.1049

 Dist from Wi = 0.2684829147813101 , Dist from Wf =7.957563150528234

 n_rows = 512 , Accuracy =0.106

 Dist from Wi = 0.3569604425847616 , Dist from Wf =7.9540853410331165

 n_rows = 1024 , Accuracy =0.11

 Dist fr

In [ ]:
np.save("accuracy_AI.npy", acc)
np.save("distance_i_AI.npy", dist_i)
np.save("distance_f_AI.npy", dist_f)

In [ ]:
#w_init ,I run  1
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,226176,192*784,192*192 + 192*784, 192*192 + 192*192 + 192*784, 1920+192*192 + 192*192 + 192*784])
num_rows_arr = np.sort(num_rows_arr)

den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      w = np.zeros((dim_w,1))
      w[0:n_rows] = wf_vec[0:n_rows]
      w[n_rows:] = w_init[n_rows:] 

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')

      net.load_state_dict(torch.load('w_affex.pth'))

      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)

              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.1038

 Dist from Wi = 0.002744307747089843 , Dist from Wf =7.962090607305482

 n_rows = 4 , Accuracy =0.1038

 Dist from Wi = 0.003881214190626746 , Dist from Wf =7.962090134276527

 n_rows = 8 , Accuracy =0.1038

 Dist from Wi = 0.005488932918213646 , Dist from Wf =7.962089188258482

 n_rows = 16 , Accuracy =0.1038

 Dist from Wi = 0.007762642058335429 , Dist from Wf =7.962087296152653

 n_rows = 32 , Accuracy =0.1038

 Dist from Wi = 0.010978065166521401 , Dist from Wf =7.96208351201194

 n_rows = 64 , Accuracy =0.1038

 Dist from Wi = 0.015525321821721708 , Dist from Wf =7.9620759437818425

 n_rows = 128 , Accuracy =0.1038

 Dist from Wi = 0.021626271722528963 , Dist from Wf =7.962061710043359

 n_rows = 256 , Accuracy =0.1038

 Dist from Wi = 0.031972579402813166 , Dist from Wf =7.9620268854314675

 n_rows = 512 , Accuracy =0.1038

 Dist from Wi = 0.052320708065069735 , Dist from Wf =7.961919172767001

 n_rows = 1024 , Accuracy =

In [ ]:
np.save("accuracy_I.npy", acc)
np.save("distance_i_I.npy", dist_i)
np.save("distance_f_I.npy", dist_f)

In [ ]:
#w_init ,I run  1
from scipy import sparse
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,226176,192*784,192*192 + 192*784, 192*192 + 192*192 + 192*784, 1920+192*192 + 192*192 + 192*784])
num_rows_arr = np.sort(num_rows_arr)

den = 10/dim_w
I = sparse.identity(dim_w)
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*inputsize))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      w = np.zeros((dim_w,1))
      w[0:-n_rows] = w_init[0:-n_rows]
      w[-n_rows:] = wf_vec[-n_rows:] 

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*inputsize]
      w1 = w1.reshape(hiddensizes[0] , inputsize)
      w2 = w[hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*inputsize+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')

      net.load_state_dict(torch.load('w_affex.pth'))

      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, inputsize)
              with torch.no_grad():
                  logps = net(img)

              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.1033

 Dist from Wi = 0.05523394501991005 , Dist from Wf =7.961899495816606

 n_rows = 4 , Accuracy =0.1095

 Dist from Wi = 0.26905801568560866 , Dist from Wf =7.957543726198588

 n_rows = 8 , Accuracy =0.1065

 Dist from Wi = 0.2877317054782213 , Dist from Wf =7.956890399887998

 n_rows = 16 , Accuracy =0.1075

 Dist from Wi = 0.4022532286958546 , Dist from Wf =7.951923459778203

 n_rows = 32 , Accuracy =0.108

 Dist from Wi = 0.43383205777709544 , Dist from Wf =7.9502631475827314

 n_rows = 64 , Accuracy =0.1085

 Dist from Wi = 0.6494271773337912 , Dist from Wf =7.935561650665154

 n_rows = 128 , Accuracy =0.1149

 Dist from Wi = 0.8355211138006975 , Dist from Wf =7.918131019284819

 n_rows = 256 , Accuracy =0.1108

 Dist from Wi = 1.1289927826715191 , Dist from Wf =7.8816413053908265

 n_rows = 512 , Accuracy =0.1065

 Dist from Wi = 1.509619664148536 , Dist from Wf =7.817668632001697

 n_rows = 1024 , Accuracy =0.1132

 Dist fr

In [ ]:
np.save("accuracy_UI.npy", acc)
np.save("distance_i_UI.npy", dist_i)
np.save("distance_f_UI.npy", dist_f)

In [ ]:
from google.colab import files
files.download('accuracy_UI.npy') 
files.download('distance_i_UI.npy')
files.download('distance_f_UI.npy')

In [ ]:
files.download('accuracy_I.npy') 
files.download('distance_i_I.npy')
files.download('distance_f_I.npy')

In [ ]:
#w_init ,A run  3
from scipy import sparse
n_runs =10
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.0498

 Dist from Wi = 0.039741766397837136 , Dist from Wf =9.117144169612043

 n_rows = 4 , Accuracy =0.0498

 Dist from Wi = 0.03998899779599565 , Dist from Wf =9.117143088574544

 n_rows = 8 , Accuracy =0.05

 Dist from Wi = 0.041442448439237244 , Dist from Wf =9.117136597690891

 n_rows = 16 , Accuracy =0.05

 Dist from Wi = 0.05754022402826041 , Dist from Wf =9.117049212333749

 n_rows = 32 , Accuracy =0.0492

 Dist from Wi = 0.10094850614009644 , Dist from Wf =9.116671904631108

 n_rows = 64 , Accuracy =0.0478

 Dist from Wi = 0.17690968444880728 , Dist from Wf =9.115514257644714

 n_rows = 128 , Accuracy =0.048

 Dist from Wi = 0.22202351027540285 , Dist from Wf =9.114527018924543

 n_rows = 256 , Accuracy =0.0494

 Dist from Wi = 0.30495464931751587 , Dist from Wf =9.112129272561383

 n_rows = 512 , Accuracy =0.0475

 Dist from Wi = 0.4305376715158752 , Dist from Wf =9.107059598532262

 n_rows = 1024 , Accuracy =0.0461

 Dist 

In [ ]:
np.save("accuracy_3.npy", acc)
np.save("distance_i_3.npy", dist_i)
np.save("distance_f_3.npy", dist_f)

In [ ]:
#winit, AI run 1
I = sparse.identity(dim_w)
n_runs =10
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs ) + I

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:] 
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.1006

 Dist from Wi = 0.03273111403659705 , Dist from Wf =9.01908498472663

 n_rows = 4 , Accuracy =0.1006

 Dist from Wi = 0.049212787296032436 , Dist from Wf =9.019010111373088

 n_rows = 8 , Accuracy =0.1006

 Dist from Wi = 0.0630456107968084 , Dist from Wf =9.01892402332135

 n_rows = 16 , Accuracy =0.1006

 Dist from Wi = 0.07530887846182159 , Dist from Wf =9.01882996071591

 n_rows = 32 , Accuracy =0.1006

 Dist from Wi = 0.08627160628615722 , Dist from Wf =9.018731756595308

 n_rows = 64 , Accuracy =0.1004

 Dist from Wi = 0.13656366539280446 , Dist from Wf =9.018110425843474

 n_rows = 128 , Accuracy =0.1002

 Dist from Wi = 0.22658142119602517 , Dist from Wf =9.01629780714088

 n_rows = 256 , Accuracy =0.1003

 Dist from Wi = 0.3148282898555762 , Dist from Wf =9.013647898351389

 n_rows = 512 , Accuracy =0.1001

 Dist from Wi = 0.46426058623115135 , Dist from Wf =9.007187540875162

 n_rows = 1024 , Accuracy =0.1009

 Dist f

In [ ]:
#winit, AI run 1
I = sparse.identity(dim_w)
n_runs =10
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs ) + I

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:] 
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.0983

 Dist from Wi = 0.015726671009795298 , Dist from Wf =9.073668261111889

 n_rows = 4 , Accuracy =0.0983

 Dist from Wi = 0.02305866286623948 , Dist from Wf =9.073652590823452

 n_rows = 8 , Accuracy =0.0983

 Dist from Wi = 0.060213419333953674 , Dist from Wf =9.07348209812717

 n_rows = 16 , Accuracy =0.0983

 Dist from Wi = 0.069376370795685 , Dist from Wf =9.073416664083627

 n_rows = 32 , Accuracy =0.0983

 Dist from Wi = 0.09778525092666067 , Dist from Wf =9.073154968675405

 n_rows = 64 , Accuracy =0.0983

 Dist from Wi = 0.12993495767297789 , Dist from Wf =9.07275150919944

 n_rows = 128 , Accuracy =0.0983

 Dist from Wi = 0.18677553746323816 , Dist from Wf =9.07175936295551

 n_rows = 256 , Accuracy =0.0983

 Dist from Wi = 0.28882104409064513 , Dist from Wf =9.069084046663745

 n_rows = 512 , Accuracy =0.0983

 Dist from Wi = 0.3991883952845662 , Dist from Wf =9.064896671546292

 n_rows = 1024 , Accuracy =0.0983

 Dist 

In [ ]:
np.save("accuracy_AI2.npy", acc)
np.save("distance_i_AI2.npy", dist_i)
np.save("distance_f_AI2.npy", dist_f)

In [ ]:
#winit, AI run 3
I = sparse.identity(dim_w)
n_runs =10
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs ) + I

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:] 
      w = np.zeros(dim_w)
     

      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.0495

 Dist from Wi = 0.02573784732717127 , Dist from Wf =9.117194457768367

 n_rows = 4 , Accuracy =0.0494

 Dist from Wi = 0.030019246612693912 , Dist from Wf =9.117181366101022

 n_rows = 8 , Accuracy =0.0495

 Dist from Wi = 0.03831346441805629 , Dist from Wf =9.117150283724191

 n_rows = 16 , Accuracy =0.0498

 Dist from Wi = 0.06876255059470655 , Dist from Wf =9.116971477919893

 n_rows = 32 , Accuracy =0.0504

 Dist from Wi = 0.08309190191320343 , Dist from Wf =9.116852140591492

 n_rows = 64 , Accuracy =0.0497

 Dist from Wi = 0.10733888444839666 , Dist from Wf =9.116598904267486

 n_rows = 128 , Accuracy =0.0491

 Dist from Wi = 0.17519154534648007 , Dist from Wf =9.115547440494437

 n_rows = 256 , Accuracy =0.0492

 Dist from Wi = 0.27101875908782436 , Dist from Wf =9.113201745245417

 n_rows = 512 , Accuracy =0.0488

 Dist from Wi = 0.4130577678706888 , Dist from Wf =9.107869152422303

 n_rows = 1024 , Accuracy =0.0521

 D

In [ ]:
np.save("accuracy_AI3.npy", acc)
np.save("distance_i_AI3.npy", dist_i)
np.save("distance_f_AI3.npy", dist_f)

In [ ]:
#w_init A=I run number 2
from scipy import sparse
I = sparse.identity(dim_w)
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)
      
      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.1007

 Dist from Wi = 0.002229636777102074 , Dist from Wf =9.019144101092259

 n_rows = 4 , Accuracy =0.1007

 Dist from Wi = 0.003153180706616471 , Dist from Wf =9.0191438254969

 n_rows = 8 , Accuracy =0.1007

 Dist from Wi = 0.004459352251144235 , Dist from Wf =9.019143274265296

 n_rows = 16 , Accuracy =0.1007

 Dist from Wi = 0.006365510069257767 , Dist from Wf =9.01914213037077

 n_rows = 32 , Accuracy =0.1007

 Dist from Wi = 0.008960603236720297 , Dist from Wf =9.019139925466124

 n_rows = 64 , Accuracy =0.1008

 Dist from Wi = 0.01472298832955942 , Dist from Wf =9.019132359665319

 n_rows = 128 , Accuracy =0.1006

 Dist from Wi = 0.09812687578567637 , Dist from Wf =9.01861055838827

 n_rows = 256 , Accuracy =0.1006

 Dist from Wi = 0.269217587888327 , Dist from Wf =9.01512546655497

 n_rows = 512 , Accuracy =0.1007

 Dist from Wi = 0.5794446534797612 , Dist from Wf =9.000511606634397

 n_rows = 1024 , Accuracy =0.1008

 Dist

In [ ]:
np.save("accuracy_I1.npy", acc)
np.save("distance_i_I1.npy", dist_i)
np.save("distance_f_I1.npy", dist_f)

In [ ]:
#w_init A=I run number 1
from scipy import sparse
I = sparse.identity(dim_w)
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)
      
      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.0497

 Dist from Wi = 0.0017267517739812833 , Dist from Wf =9.117230623159402

 n_rows = 4 , Accuracy =0.0497

 Dist from Wi = 0.0024419205797025813 , Dist from Wf =9.117230459661045

 n_rows = 8 , Accuracy =0.0497

 Dist from Wi = 0.0034534131730317216 , Dist from Wf =9.117230132638133

 n_rows = 16 , Accuracy =0.0497

 Dist from Wi = 0.004883833131892074 , Dist from Wf =9.117229478614718

 n_rows = 32 , Accuracy =0.0497

 Dist from Wi = 0.006906732635710126 , Dist from Wf =9.11722817058954

 n_rows = 64 , Accuracy =0.0497

 Dist from Wi = 0.009771478709027363 , Dist from Wf =9.117225550338821

 n_rows = 128 , Accuracy =0.0498

 Dist from Wi = 0.013209762455709095 , Dist from Wf =9.117221217000443

 n_rows = 256 , Accuracy =0.0498

 Dist from Wi = 0.018306320348713226 , Dist from Wf =9.117212408197064

 n_rows = 512 , Accuracy =0.0498

 Dist from Wi = 0.024330710601050368 , Dist from Wf =9.11719832152777

 n_rows = 1024 , Accuracy =

In [ ]:
np.save("accuracy_I1.npy", acc)
np.save("distance_i_I1.npy", dist_i)
np.save("distance_f_I1.npy", dist_f)

In [ ]:
#w_init A=I run number 2
from scipy import sparse
I = sparse.identity(dim_w)
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)
      
      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.1052

 Dist from Wi = 0.003822708623225961 , Dist from Wf =9.093001913493021

 n_rows = 4 , Accuracy =0.1052

 Dist from Wi = 0.0054060583947012865 , Dist from Wf =9.09300110999782

 n_rows = 8 , Accuracy =0.1052

 Dist from Wi = 0.007645391564762392 , Dist from Wf =9.09299950290754

 n_rows = 16 , Accuracy =0.1052

 Dist from Wi = 0.010865902174918915 , Dist from Wf =9.092996224790753

 n_rows = 32 , Accuracy =0.1052

 Dist from Wi = 0.015328690725463709 , Dist from Wf =9.092989796713157

 n_rows = 64 , Accuracy =0.1055

 Dist from Wi = 0.025840978279708854 , Dist from Wf =9.092965998821724

 n_rows = 128 , Accuracy =0.1047

 Dist from Wi = 0.08989761502434604 , Dist from Wf =9.092558321542466

 n_rows = 256 , Accuracy =0.1047

 Dist from Wi = 0.24125692787738195 , Dist from Wf =9.089801620835953

 n_rows = 512 , Accuracy =0.1052

 Dist from Wi = 0.5310215179739362 , Dist from Wf =9.077483933247354

 n_rows = 1024 , Accuracy =0.1066

In [ ]:
np.save("accuracy_I2.npy", acc)
np.save("distance_i_I2.npy", dist_i)
np.save("distance_f_I2.npy", dist_f)

In [ ]:
#w_init A=I run number 3
from scipy import sparse
I = sparse.identity(dim_w)
n_runs =1
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,32768,65536,131072,200000])
den = 10/dim_w
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))
rvs = stats.norm().rvs

wi = torch.load('weights_i.pth')
wi1 = np.asarray(wi['0.weight'].resize(1,hiddensizes[0]*784))
wi2 = np.asarray(wi['2.weight'].resize(1,hiddensizes[0]*hiddensizes[1]))
wi3 = np.asarray(wi['4.weight'].resize(1,hiddensizes[1]*hiddensizes[2]))
wi4 = np.asarray(wi['6.weight'].resize(1,hiddensizes[2]*10))
w_init = np.concatenate((wi1,wi2,wi3,wi4), axis = 1)
w_init = w_init.T

w_init.ravel()
w_init.squeeze()

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)
      
      w_diff = csr_matrix(wf_vec - w_init)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ w_init

      #test performance of this reinitialization
      w1= w[0:hiddensizes[0]*784]
      w1 = w1.reshape(hiddensizes[0] , 784)
      w2 = w[hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784]
      w2 = w2.reshape(hiddensizes[1],hiddensizes[0])
      w3 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784:hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]]
      w3 = w3.reshape(hiddensizes[2],hiddensizes[1])
      w4 = w[hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2]: hiddensizes[1]*hiddensizes[0]+hiddensizes[0]*784+hiddensizes[1]*hiddensizes[2] +hiddensizes[2]*10]
      w4 = w4.reshape(10 , hiddensizes[2])

      wf =torch.load('weights_f.pth')
      wf['0.weight'] = torch.from_numpy(w1)
      wf['2.weight'] = torch.from_numpy(w2)
      wf['4.weight'] = torch.from_numpy(w3)
      wf['6.weight'] = torch.from_numpy(w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct_count, all_count = 0, 0
      for images,labels in valloader:
          for i in range(len(labels)):
              img = images[i].view(1, 784)
              with torch.no_grad():
                  logps = net(img)


              ps = torch.exp(logps)
              probab = list(ps.numpy()[0])
              pred_label = probab.index(max(probab))
              true_label = labels.numpy()[i]
              if(true_label == pred_label):
                    correct_count += 1
              all_count += 1

      acc[i_out,i_in] = correct_count/all_count
      dist_i[i_out,i_in] = np.linalg.norm(w-w_init)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in]))    

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


outer iteration number: 1

 n_rows = 2 , Accuracy =0.1026

 Dist from Wi = 0.0008038168116411688 , Dist from Wf =9.029198878006

 n_rows = 4 , Accuracy =0.1026

 Dist from Wi = 0.0011363645008773619 , Dist from Wf =9.029198842277314

 n_rows = 8 , Accuracy =0.1026

 Dist from Wi = 0.0016071359001997101 , Dist from Wf =9.029198770755933

 n_rows = 16 , Accuracy =0.1026

 Dist from Wi = 0.002272872691635544 , Dist from Wf =9.029198627716413

 n_rows = 32 , Accuracy =0.1026

 Dist from Wi = 0.0032143787797732025 , Dist from Wf =9.029198341628968

 n_rows = 64 , Accuracy =0.1025

 Dist from Wi = 0.0058374405807338344 , Dist from Wf =9.029197026812176

 n_rows = 128 , Accuracy =0.1027

 Dist from Wi = 0.07222301399849446 , Dist from Wf =9.028910059409734

 n_rows = 256 , Accuracy =0.1026

 Dist from Wi = 0.15809044016921356 , Dist from Wf =9.02781482073479

 n_rows = 512 , Accuracy =0.1026

 Dist from Wi = 0.32465767305209314 , Dist from Wf =9.023360262128088

 n_rows = 1024 , Accuracy =0.1

In [ ]:
np.save("accuracy_I3.npy", acc)
np.save("distance_i_I3.npy", dist_i)
np.save("distance_f_I3.npy", dist_f)